In [2]:
import argparse
import numpy as np
import json
import random
import keras
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [3]:
batch_size=256
top_n=5
model_type="l2ac"
set_mode="train1"
db="amazon"
ncls=9

out_dir="../runs/l2ac_5_9/0/"

output_fn=out_dir+"train.h5"
eval_fn=out_dir+"eval.h5"
data=np.load("../"+db+"/data/"+set_mode+"_idx.npz")

In [4]:
train_X0=np.repeat(data['train_X0'], ncls, axis=0)
train_X1=data['train_X1'][:,-ncls:,-top_n:].reshape(-1, top_n)
train_Y=data['train_Y'][:,-ncls:].reshape(-1,)
valid_X0=np.repeat(data['valid_X0'], 2, axis=0) #the validation data is balanced.
valid_X1=data['valid_X1'][:,-2:,-top_n:].reshape(-1, top_n)
valid_Y=data['valid_Y'][:,-2:].reshape(-1,)

In [5]:
print(data.files)

['train_rep', 'train_Y', 'valid_X1', 'train_X0', 'valid_Y', 'train_X1', 'valid_X0']


In [6]:
query=keras.Input(shape=(data['train_rep'].shape[1],), dtype="float32" )
x_rep=query
x1=keras.Input(shape=(top_n, data['train_rep'].shape[1]), dtype="float32" )
x1_rep=x1

In [7]:
print(data['train_rep'].shape)
print(data['train_rep'])

(100000, 512)
[[0.         0.00376278 0.         ... 0.         0.04510138 0.0141681 ]
 [0.         0.         0.03986862 ... 0.         0.08018569 0.01789098]
 [0.         0.05254612 0.02741214 ... 0.         0.05547189 0.00843837]
 ...
 [0.         0.02203734 0.02549556 ... 0.47996774 0.00141776 0.01909813]
 [0.         0.         0.         ... 0.03336517 0.         0.00200262]
 [0.         0.         0.0822084  ... 0.00569493 0.00078439 0.01160259]]


In [9]:
print(query)
print(x1)

Tensor("input_1:0", shape=(?, 512), dtype=float32)
Tensor("input_2:0", shape=(?, 5, 512), dtype=float32)


In [10]:
#"many_lstm_simspace"
x_rep=keras.layers.RepeatVector(train_X1.shape[1] )(x_rep)
if "abssub" in model_type:
    x_sub=keras.layers.Subtract()([x_rep, x1_rep])
    x_rep=keras.layers.Lambda(lambda x: K.abs(x) )(x_sub)
elif "sum" in model_type:
    x_rep=keras.layers.Add()([x_rep, x1_rep])
else:
    x_sub=keras.layers.Subtract()([x_rep, x1_rep])
    x_abs=keras.layers.Lambda(lambda x: K.abs(x) )(x_sub)
    x_add=keras.layers.Add()([x_rep, x1_rep])
    x_rep=keras.layers.Concatenate(axis=-1)([x_abs, x_add])

In [32]:
a = list(range(16))
b = list(range(32))
dsa = tf.data.Dataset.range(16).window(1,shift=1,drop_remainder=True)
dsa = dsa.flat_map(lambda w: w.batch(1))
dsb = tf.data.Dataset.from_tensor_slices(b).window(1,shift=1,drop_remainder=True)
dsb = dsb.flat_map(lambda w: w.batch(1))
print(dsa)
print(dsb)
print(type(dsa),type(dsb))

<DatasetV1Adapter shapes: (?,), types: tf.int64>
<DatasetV1Adapter shapes: (?,), types: tf.int32>
<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'> <class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>


In [85]:
input1 = keras.layers.Input(shape=(16,))
x1 = keras.layers.Dense(8, activation='relu')(input1)
input2 = keras.layers.Input(shape=(32,))
x2 = keras.layers.Dense(8, activation='relu')(input2)
# Equivalent to subtracted = keras.layers.subtract([x1, x2])
subtracted = keras.layers.Subtract()([x1, x2])

out = keras.layers.Dense(4)(subtracted)
model = keras.models.Model(inputs=[input1, input2], outputs=out)
model.compile(optimizer='adam',loss=keras.losses.categorical_crossentropy)
model.summary()

#print(in_put)
dsa = np.random.randint(100,size=(16,))
dsb = np.random.randint(100,size=(32,))
print(dsa.shape)
print(dsa)
print(dsb.shape)
print(dsb)
print(type(dsa))
print(type(dsb))

'''print(in_put.shape)
print(in_put[0].shape)
print(in_put[1].shape)'''

history = model.predict([dsa.numpy(),dsb.numpy()])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_121 (InputLayer)          (None, 16)           0                                            
__________________________________________________________________________________________________
input_122 (InputLayer)          (None, 32)           0                                            
__________________________________________________________________________________________________
dense_178 (Dense)               (None, 8)            136         input_121[0][0]                  
__________________________________________________________________________________________________
dense_179 (Dense)               (None, 8)            264         input_122[0][0]                  
__________________________________________________________________________________________________
subtract_6

AttributeError: 'numpy.ndarray' object has no attribute 'numpy'